Author Yizhou Li, e-mail: lyzpp2000@163.com

Problem Statement:

Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling<br>
or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences<br>
price negotiations than the number of bedrooms or a white-picket fence.<br>
<br>
With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition<br>
challenges you to predict the final price of each home.

# 1. Data Preprocessing

In [ ]:
# Step1: Loading necessary libraries and datasets.

# Step2: Drop duplicate labels and columns and rows with few information, and handle missing values.
drop_duplicates(train_df, test_df)
drop_columns_missing_above_80_percent(train_df, test_df)
fill_na(train_df, test_df)

print(train_df.head())
print(train_df.info())
print(test_df.head())
print(test_df.info())

# Step3: Encode categorical values into numerical type.

# Step4: Split dataset into training and testing sets.

# Step5: Feature scaling.

# 2. Define Important Functions

In [ ]:
# todo

In [ ]:
fig, axs = plt.subplots(30, 3)
my_lst = [col for col in train_df.columns]
fig.set_size_inches(25, 200)
row = 30
col = 3
y = train_df["SalePrice"]
for i in range(row):
    for j in range(col):
        if len(my_lst) == 0:
            break
        x = train_df[my_lst.pop()]
        axs[i, j].scatter(x, y)
        axs[i, j].set_xlabel(x.name)
        axs[i, j].set_ylabel("SalePrice")
        axs[i, j].set_title(x.name + " vs SalePrice")

In [60]:
x_data = train_df[['LotFrontage', 'LotArea', 'MasVnrArea', 'TotalBsmtSF', 'GrLivArea',
                    'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 
                    'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 
                    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']]
y_data = train_df["SalePrice"]
# my_train_lst = [col for col in train_df.columns]
# my_test_lst = [col for col in test_df.columns]
# my_train_lst.pop()
# my_test_lst.pop()

# x_data = train_df[my_train_lst]
# test_data = test_df[my_test_lst]
test_data = test_df[ ['LotFrontage', 'LotArea', 'MasVnrArea', 'TotalBsmtSF', 'GrLivArea',
                    'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 
                    'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 
                    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']]
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data)

# 5. Models building, evaluation, and predicting


In [61]:
model = LinearRegression()
model2 = Lasso()
model3 = Ridge()
model4 = DecisionTreeRegressor()
model5 = RandomForestRegressor()

model.fit(X=x_train, y=y_train)
model2.fit(X=x_train, y=y_train)
model3.fit(X=x_train, y=y_train)
model4.fit(X=x_train, y=y_train)
model5.fit(X=x_train, y=y_train)

print("the accuracy score using with LinearRegression() model ", model.score(x_test, y_test))
print("the accuracy score using with Lasso() model ", model2.score(x_test, y_test))
print("the accuracy score using with Ridge() model ", model3.score(x_test, y_test))
print("the accuracy score using with DecisionTreeRegressor() model ", model4.score(x_test, y_test))
print("the accuracy score using with RandomForestRegressor() model ", model5.score(x_test, y_test))

model.fit(X=x_train, y=y_train)
result = model.predict(test_data)
temp = test_df
temp.reset_index(inplace=True)
metric = pd.Series(result, name = 'SalePrice')
final_metric = pd.concat([temp["Id"], metric], axis = 1)
final_metric.to_csv("submission.csv",index =False)

the accuracy score using with LinearRegression() model  0.699274404306837
the accuracy score using with Lasso() model  0.699255842859624
the accuracy score using with Ridge() model  0.6988298969020352
the accuracy score using with DecisionTreeRegressor() model  0.5258261751040558
the accuracy score using with RandomForestRegressor() model  0.647976119535729


ValueError: cannot insert level_0, already exists